In [ ]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.insert(0, "../src/")
sys.path.insert(0, "../../../src/")
sys.path.insert(0, "../../../projects/skimpy_bench/src/")
sys.path.insert(0, "../../../projects/skimpy_blox/")

In [ ]:
import random
import numpy as np
import skimpy
import skimpy_3d
import ipywebrtc
from skimpy_blox import minecraft, colors
from skimpy import functional as F
from pythreejs import *
from IPython.display import display

In [ ]:
%%time
# level = minecraft.SkimpyMinecraftLevel.load("../data/sample_minecraft_world.pickle.gz")
level = minecraft.SkimpyMinecraftLevel.load("../data/Hogwarts1.0.pickle.gz")

In [ ]:
%%time
t = level.megatensor().min(1).to(float)

In [ ]:
F.sum(t)

In [ ]:
t.shape

In [ ]:
def blur(tensor):
    G_x = (1 / 64.0) * skimpy.Tensor.from_list([[[1, 6, 15, 20, 15, 6, 1]]]).to(float)
    G_y = (1 / 64.0) * skimpy.Tensor.from_list([[[1], [6], [15], [20], [15], [6], [1]]]).to(float)
    G_z = (1 / 64.0) * skimpy.Tensor.from_list([[[1]], [[6]], [[15]], [[20]], [[15]], [[6]], [[1]]]).to(float)
    tensor = F.conv_3d(tensor, G_x, padding=(3, 0, 0), fill=0.0)
    tensor = F.conv_3d(tensor, G_y, padding=(0, 3, 0), fill=0.0)  
    tensor = F.conv_3d(tensor, G_z, padding=(0, 0, 3), fill=0.0)
    return tensor

In [ ]:
%%time
for i in range(4):
    t = blur(t)

In [ ]:
%%time
avg_kernel = skimpy.Tensor.from_list([[[1, 1], [1, 1]], [[1, 1], [1, 1]]]).to(float) / 8
density_lattice = F.conv_3d(
    t,
    avg_kernel,
    padding=1,
)

In [ ]:
%%time
mesh = skimpy_3d.marching_cubes(density_lattice._tensor, 0.1)

In [ ]:
geometry = BufferGeometry(
        attributes={
            "position": BufferAttribute(
                np.array(mesh.positions, dtype='float32').reshape(-1, 3),
                normalized=False,
            ),
            "index": BufferAttribute(
                np.array(mesh.triangles, dtype='uint32'),
                normalized=False,
            ),
        },
    )

geometry.exec_three_obj_method("computeVertexNormals")

In [ ]:
js_mesh = Mesh(
    geometry=geometry,
    material=MeshLambertMaterial(color="lightblue"),
    position=[0, 0, 0],
)

In [ ]:
js_mesh

In [ ]:
camera_pos = level.xyz_to_skimpy_col(level.dense_dimensions())
camera = PerspectiveCamera(position=camera_pos, lookAt=(0,0,0), fov=20)
up = [0, 0, 0]
up[level.column_order[1]] = 1
camera.up = tuple(up)

In [ ]:
light = PointLight(
    color='#ffffff',
    position=level.xyz_to_skimpy_col(tuple(e / 2 for e in level.dense_dimensions())),
)
scene = Scene(children=[js_mesh, camera, light])

renderer = Renderer(
    camera=camera, 
    background="white", 
    background_opacity=1,
    scene=scene,
    controls=[OrbitControls(controlling=camera)],
    width=984,
    height=984,
)

display(renderer)